# Creating target data files
### NOTE: This notebook does not include Scaling/Normalization of data. That is in CSL_create_targets_norm.ipynb

In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, HTML

In [2]:
pd.options.display.max_columns = 30
# Note: Name currently assumes updated symbolic link
os.chdir('/MFMDatasets/MFM_bopf/data/csl')
df = pd.read_csv('CSL_d6_fields.csv', index_col=0)
print(df.shape)
display(df.head())

(185413, 208)


,trans_loss,transfus_yes,transfus_all,transfus_hyster,MomNearMiss,Hysterectomy,Momdeath,MomICU,Postbleed,high_MomLOS,Bloodproduct,Posttransfus,EBLoss,high_EBLoss,hemorrhage,...,TrialLabor,UnspecHBP,Urupture,uscar,version9,vertex,Activeherpes,iufd9,high_BMI,high_Age,high_Gravidity,high_height,intra_abruptio,intra_previa,pre_PROM
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,0,0,0,0,0,0,0,3,0,0,350.0,0,0,...,0,0,0,1,0,9,0,0,0,0,2,2,0,0,0
41-00003,0,0,0,0,0,0,0,0,0,2,0,0,250.0,0,0,...,1,0,0,0,0,1,0,0,0,0,3,2,0,0,0
41-00004,0,0,0,0,0,0,0,0,0,1,0,0,250.0,0,0,...,1,0,0,0,0,1,0,0,0,0,4,3,0,0,0
41-00005,1,1,1,1,0,0,0,0,0,2,0,1,350.0,0,1,...,1,0,0,0,0,1,0,0,0,0,3,2,0,0,0
41-00006,0,0,0,0,0,0,0,0,0,2,0,0,450.0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,3,0,0,0


In [3]:
# Drop all extra targets and collinear data fields
drop_list = ['transfus_all','transfus_hyster',\
             'MomNearMiss','Hysterectomy','Momdeath','MomICU','Postbleed',\
             'high_MomLOS','Bloodproduct','Posttransfus','EBLoss','high_EBLoss',\
             'hemorrhage','postpartum_hemorrhage9']
df.drop(drop_list, axis=1, inplace=True)
display(df.head())

,trans_loss,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,Admreason,AdmSBP,Alcohol,Analgesia,Anteabruption,...,TrialLabor,UnspecHBP,Urupture,uscar,version9,vertex,Activeherpes,iufd9,high_BMI,high_Age,high_Gravidity,high_height,intra_abruptio,intra_previa,pre_PROM
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,0,8,8,8,99.0,83,0.0,77,1,150.0,0,7,0,...,0,0,0,1,0,9,0,0,0,0,2,2,0,0,0
41-00003,0,0,0,6,3,2,2.9,77,70.0,1,3,105.0,0,8,0,...,1,0,0,0,0,1,0,0,0,0,3,2,0,0,0
41-00004,0,0,0,5,3,2,3.3,65,70.0,1,3,122.0,0,8,0,...,1,0,0,0,0,1,0,0,0,0,4,3,0,0,0
41-00005,1,1,0,11,2,3,4.0,77,80.0,77,4,138.0,0,8,0,...,1,0,0,0,0,1,0,0,0,0,3,2,0,0,0
41-00006,0,0,0,11,1,3,5.0,98,100.0,77,4,134.0,0,8,0,...,1,0,0,0,0,1,0,0,0,0,1,3,0,0,0


## Break up file into groups of columns used for different versions of output files 

In [4]:
# Save trans_loss and tranfus_yes for later
tl_series = df['trans_loss']
ty_series = df['transfus_yes']

In [5]:
# Read in variables categorized by "timing" (pre, intra, post, etc)
varTimingPath = 'V3_d5_code_timing.csv'
var_times_df = pd.read_csv(varTimingPath, delimiter='\t')
var_times_df

,Code,timing
0,delivery,drop
1,fever9,drop
2,ga,drop
3,Lac_Unkn,drop
4,onepregflag,index
...,...,...
243,MomID,target
244,Postbleed,target
245,postpartum_hemorrhage9,target
246,Posttransfus,target


In [6]:
# Save off "pre" and "intra" columns
pre_df = df[var_times_df[var_times_df['timing'] == 'pre']['Code'].to_list()]
display(pre_df)
intra_df = df[var_times_df[var_times_df['timing'] == 'intra']['Code'].to_list()]
display(intra_df)

,Activeherpes,Admreason,Alcohol,Anteabruption,Anteanemia,Anteasthma,Antebleed3,AnteCHBP,Antechorio,Antefetaldth,Antefetdistress,AnteGBS,AnteGDM,Antehospital,AnteLGA,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52-07248,0,2,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
52-07249,0,5,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,chorio,...,Meconium,Momseizure,MthInd_AROM,MthInd_Oxy,Operative,Presentdel,Prolapse,ROM,ROMmeth,SE_pre,Shoulder,spontlabor,TrialLabor,Urupture,vertex
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,99.0,83,0.0,77,150.0,7,1,38.2,0,0,0,...,1,0,0,0,0,88,0,0,1,0,0,1,0,0,9
41-00003,0,6,3,2,2.9,77,70.0,1,105.0,8,88,39.0,0,0,0,...,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1
41-00004,0,5,3,2,3.3,65,70.0,1,122.0,8,88,39.1,0,0,0,...,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1
41-00005,0,11,2,3,4.0,77,80.0,77,138.0,8,88,38.4,0,0,0,...,4,0,0,0,0,1,0,0,2,0,0,1,1,0,1
41-00006,0,11,1,3,5.0,98,100.0,77,134.0,8,88,39.6,0,0,0,...,1,0,0,0,0,1,0,0,2,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,8,8,8,77.0,74,70.0,1,124.4,5,4,37.1,0,0,0,...,5,0,0,1,1,1,0,0,1,0,1,0,1,0,1
52-07248,0,8,8,8,99.0,74,0.0,2,124.4,5,0,24.6,1,1,0,...,1,0,0,1,0,2,0,0,8,0,0,0,1,0,0
52-07249,0,8,8,8,66.0,74,0.0,77,124.4,9,0,28.7,0,0,0,...,1,0,0,0,0,88,0,0,2,0,0,0,0,0,9


In [7]:
ty_PI = pd.concat([intra_df,pre_df], axis=1)
ty_PI.insert(0, 'transfus_yes', ty_series)
print(ty_PI.shape)
ty_PI.head()

(185413, 193)


,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,99.0,83,0.0,77,150.0,7,1,38.2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,6,3,2,2.9,77,70.0,1,105.0,8,88,39.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,5,3,2,3.3,65,70.0,1,122.0,8,88,39.1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,11,2,3,4.0,77,80.0,77,138.0,8,88,38.4,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,11,1,3,5.0,98,100.0,77,134.0,8,88,39.6,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
ty_Pre = pd.concat([pre_df], axis=1)
ty_Pre.insert(0, 'transfus_yes', ty_series)
print(ty_Pre.shape)
ty_Pre.head()

(185413, 118)


,transfus_yes,Activeherpes,Admreason,Alcohol,Anteabruption,Anteanemia,Anteasthma,Antebleed3,AnteCHBP,Antechorio,Antefetaldth,Antefetdistress,AnteGBS,AnteGDM,Antehospital,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
tl_PI = pd.concat([intra_df,pre_df], axis=1)
tl_PI.insert(0, 'trans_loss', tl_series)
print(tl_PI.shape)
tl_PI.head()

(185413, 193)


,trans_loss,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,99.0,83,0.0,77,150.0,7,1,38.2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,6,3,2,2.9,77,70.0,1,105.0,8,88,39.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,5,3,2,3.3,65,70.0,1,122.0,8,88,39.1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,11,2,3,4.0,77,80.0,77,138.0,8,88,38.4,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,11,1,3,5.0,98,100.0,77,134.0,8,88,39.6,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
tl_Pre = pd.concat([pre_df], axis=1)
tl_Pre.insert(0, 'trans_loss', tl_series)
print(tl_Pre.shape)
tl_Pre.head()

(185413, 118)


,trans_loss,Activeherpes,Admreason,Alcohol,Anteabruption,Anteanemia,Anteasthma,Antebleed3,AnteCHBP,Antechorio,Antefetaldth,Antefetdistress,AnteGBS,AnteGDM,Antehospital,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Use "out_flag" variable to control whether files are written or not

In [11]:
out_flag = True

from datetime import datetime
datestr = datetime.today().strftime('%Y-%m-%d')

In [12]:
# Write out transfus_yes for "pre_intra"
filename = 'MFM_CSL_d6_transfus_yes_Pre-Intra_' + datestr + '.csv'
linkname = 'CSL_ty_PI.csv'
if out_flag:
    ty_PI.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print(f'File not written: {filename}')

In [13]:
# Write out transfus_yes for "pre" only
filename = 'MFM_CSL_d6_transfus_yes_Pre_' + datestr + '.csv'
linkname = 'CSL_ty_Pre.csv'
if out_flag:
    ty_Pre.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print(f'File not written: {filename}')

In [14]:
# Write out trans_loss for "pre_intra"
filename = 'MFM_CSL_d6_trans_loss_Pre-Intra_' + datestr + '.csv'
linkname = 'CSL_tl_PI.csv'
if out_flag:
    tl_PI.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print(f'File not written: {filename}')

In [15]:
# Write out trans_loss for "pre" only
filename = 'MFM_CSL_d6_trans_loss_Pre_' + datestr + '.csv'
linkname = 'CSL_tl_Pre.csv'
if out_flag:
    tl_Pre.to_csv(filename, header=True)
    os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print(f'File not written: {filename}')

## Bin continuous variables 
### Note - only applicable to PI

In [16]:
# Create new copy of intra_df for binning
bdf = intra_df.copy(deep=True)

In [17]:
# Re-bin Admcontract for DL
pd.options.display.max_rows = 100
bdf['Admcontract'].value_counts(sort=False, dropna=False).sort_index()

0.00      9285
0.06         1
0.07        31
0.08        43
0.09         7
0.10        44
0.20       334
0.23         1
0.25         2
0.30       240
0.40        54
0.43         1
0.50       253
0.60        47
0.70       299
0.80        98
0.90        37
1.00      8474
1.10       160
1.20        88
1.30       520
1.40       480
1.50       528
1.60        29
1.70       825
1.80       650
1.90        44
2.00     15253
2.10       136
2.20      1446
2.25         2
2.30        14
2.40       148
2.50      3210
2.70       379
2.90      4087
3.00     11466
3.10       836
3.20         2
3.30      6716
3.50       119
3.60      1881
3.70         2
3.80         4
4.00     14776
4.10         1
4.20         4
4.30         2
4.40      2801
4.50        22
4.60         3
4.70         2
4.80         1
5.00      7356
5.30         5
5.40         1
5.50         9
5.70      1661
5.90         1
6.00       139
6.25         2
6.50         1
6.60         1
6.70      1135
7.00        36
7.70         2
8.00      

In [18]:
bdf.loc[(bdf['Admcontract'] < 1) | (bdf['Admcontract'] == 88) | (bdf['Admcontract'] == 99), 'temp'] = 0
bdf.loc[(bdf['Admcontract'] >= 1) & (bdf['Admcontract'] <= 3), 'temp'] = 1
bdf.loc[(bdf['Admcontract'] > 3) & (bdf['Admcontract'] <= 5), 'temp'] = 2
bdf.loc[(bdf['Admcontract'] > 5) & (bdf['Admcontract'] <= 10), 'temp'] = 3
bdf.loc[(bdf['Admcontract'] > 10) & (bdf['Admcontract'] < 88), 'temp'] = 4
bdf['Admcontract'] = bdf['temp'].astype('Int8')
bdf['Admcontract'].value_counts(dropna=False)

0      70729
1      47939
2      34528
4      27451
3       4766
NaN        0
Name: Admcontract, dtype: Int64

In [19]:
bdf['Admefface'].value_counts(sort=False).sort_index()

0.00      58282
0.20          3
0.25          1
0.30          4
0.40          5
0.50         35
0.60         13
0.70         20
0.80         44
0.90         36
1.00         86
2.00         21
3.00          3
4.00          1
5.00          1
8.00          1
9.00          2
10.00       629
15.00         2
20.00      1661
24.00         1
25.00       467
30.00      2752
35.00         2
40.00      2480
45.00         2
50.00     16995
52.00         1
55.00         2
60.00      9280
65.00        18
70.00     18575
72.00         1
75.00       863
80.00     26267
82.00         1
83.00         1
85.00        70
90.00     20620
95.00        58
96.00         1
99.00         2
100.00    26104
Name: Admefface, dtype: int64

In [20]:
# Re-bin Admefface 
if 'temp' in bdf.columns:
    bdf.drop('temp', axis=1, inplace=True)
bdf.loc[(bdf['Admefface'] < 10), 'temp'] = 0
bdf.loc[(bdf['Admefface'] >= 10) & (bdf['Admefface'] < 29.9), 'temp'] = 1
bdf.loc[(bdf['Admefface'] >= 30) & (bdf['Admefface'] < 59.9), 'temp'] = 2
bdf.loc[(bdf['Admefface'] >= 60) & (bdf['Admefface'] < 79.9), 'temp'] = 3
bdf.loc[(bdf['Admefface'] >= 80), 'temp'] = 4
bdf['Admefface'] = bdf['temp'].astype('Int8')
bdf['Admefface'].value_counts(dropna=False)

4      73124
0      58558
3      28737
2      22234
1       2760
NaN        0
Name: Admefface, dtype: Int64

In [21]:
pd.options.display.max_rows = None
bdf['AdmSBP'].value_counts(sort=True, dropna=False).sort_index()

40.0         1
42.0         1
51.0         1
54.0         1
56.0         1
58.0         3
59.0         3
60.0         2
61.0         1
62.0         4
63.0         4
64.0         2
65.0         3
66.0         2
67.0         3
68.0         3
69.0         4
70.0         7
71.0         6
72.0         7
73.0         1
74.0         7
75.0         6
76.0         8
77.0        18
78.0        14
79.0        14
80.0        25
81.0        15
82.0        32
83.0        47
84.0        51
85.0        42
86.0        58
87.0        85
88.0        82
89.0        98
90.0       200
91.0       188
92.0       234
93.0       289
94.0       308
95.0       361
96.0       448
97.0       562
98.0       598
99.0       641
100.0      970
101.0      946
102.0     1100
103.0     1246
104.0     1423
105.0     1573
106.0     1774
107.0     1923
108.0     2021
109.0     2165
110.0     3111
111.0     2894
112.0     3244
113.0     3291
114.0     3422
115.0     3582
116.0     3652
117.0     3724
118.0     3825
119.0     

In [22]:
# Re-bin AdmSBP 
if 'temp' in bdf.columns:
    bdf.drop('temp', axis=1, inplace=True)
bdf.loc[(bdf['AdmSBP'] <= 90), 'temp'] = 0
bdf.loc[(bdf['AdmSBP'] > 90) & (bdf['AdmSBP'] < 140), 'temp'] = 1
bdf.loc[(bdf['AdmSBP'] >= 140) & (bdf['AdmSBP'] < 160), 'temp'] = 2
bdf.loc[(bdf['AdmSBP'] >= 160) & (bdf['AdmSBP'] < 210), 'temp'] = 3
bdf.loc[(bdf['AdmSBP'] >= 210), 'temp'] = 4
bdf['AdmSBP'] = bdf['temp'].astype('Int8')
bdf['AdmSBP'].value_counts(dropna=False)

1      164143
2       17907
3        2463
0         862
4          38
NaN         0
Name: AdmSBP, dtype: Int64

In [23]:
bdf['AdmDBP'].value_counts(sort=True, dropna=False).sort_index()

20        15
22         3
23         4
24         4
26         3
27         2
28         9
29         7
30         7
31        13
32        11
33        12
34        16
35        23
36        28
37        33
38        35
39        36
40        85
41        78
42        81
43       106
44       132
45       164
46       190
47       219
48       303
49       313
50       511
51       482
52       577
53       682
54       783
55       969
56      1098
57      1260
58      1438
59      1627
60      2294
61      2175
62      2668
63      2834
64      3348
65      3492
66      3946
67      4222
68      4627
69      4810
70      5321
71      5186
72      5424
73      5258
74     50178
75      4938
76      4984
77      4701
78      5094
79      4552
80      4556
81      3905
82      3563
83      3245
84      3103
85      2897
86      2698
87      2476
88      2361
89      1971
90      1846
91      1341
92      1302
93      1066
94      1053
95       912
96       780
97       744
98       614

In [24]:
# Re-bin AdmDBP 
if 'temp' in bdf.columns:
    bdf.drop('temp', axis=1, inplace=True)
bdf.loc[(bdf['AdmDBP'] < 30), 'temp'] = 0
bdf.loc[(bdf['AdmDBP'] >= 30) & (bdf['AdmDBP'] < 90), 'temp'] = 1
bdf.loc[(bdf['AdmDBP'] >= 90) & (bdf['AdmDBP'] < 110), 'temp'] = 2
bdf.loc[(bdf['AdmDBP'] >= 110), 'temp'] = 3
bdf['AdmDBP'] = bdf['temp'].astype('Int8')
bdf['AdmDBP'].value_counts(dropna=False)

1      172139
2       12444
3         783
0          47
NaN         0
Name: AdmDBP, dtype: Int64

In [25]:
bdf['BESTGA'].value_counts(sort=True, dropna=False).sort_index()

23.00      152
23.10       21
23.14        8
23.20        3
23.29       12
23.30       16
23.40       14
23.43        8
23.50        4
23.57       10
23.60       24
23.70       22
23.71       14
23.86       12
23.90       27
24.00      169
24.10       23
24.14       13
24.20        5
24.29       24
24.30       26
24.40       29
24.43       14
24.50        7
24.57       11
24.60       18
24.70       21
24.71       15
24.86       18
24.90       23
25.00      171
25.10       17
25.14       14
25.20        7
25.29       12
25.30       31
25.40       39
25.43       16
25.50        6
25.57       12
25.60       22
25.70       22
25.71       18
25.86       13
25.90       14
26.00      176
26.10       23
26.14       13
26.20        9
26.29       15
26.30       26
26.40       28
26.43       13
26.50       15
26.57       15
26.60       23
26.70       18
26.71       17
26.86       15
26.90       19
27.00      164
27.10       31
27.14       14
27.20        7
27.29       18
27.30       26
27.40     

# Re-bin BESTGA 
if 'temp' in bdf.columns:
    bdf.drop('temp', axis=1, inplace=True)
bdf.loc[(bdf['BESTGA'] <= 24), 'temp'] = 0
bdf.loc[(bdf['BESTGA'] > 24) & (bdf['BESTGA'] < 28), 'temp'] = 1
bdf.loc[(bdf['BESTGA'] >= 28) & (bdf['BESTGA'] < 32), 'temp'] = 2
bdf.loc[(bdf['BESTGA'] >= 32) & (bdf['BESTGA'] < 36), 'temp'] = 3
bdf.loc[(bdf['BESTGA'] >= 36) & (bdf['BESTGA'] < 38), 'temp'] = 5
bdf.loc[(bdf['BESTGA'] >= 38) & (bdf['BESTGA'] < 42), 'temp'] = 5
bdf.loc[(bdf['BESTGA'] >= 42), 'temp'] = 6
bdf['BESTGA'] = bdf['temp'].astype('Int8')
bdf['BESTGA'].value_counts(dropna=False).sort_index()

In [26]:
if 'temp' in bdf.columns:
    bdf.drop('temp', axis=1, inplace=True)
pd.options.display.max_rows = 30
for col in ['AdmSBP','AdmDBP','BESTGA','Admcontract','Admefface']:
    print(f'{col}.value_counts():')
    print(bdf[col].value_counts(dropna=False).to_string() + '\n')

AdmSBP.value_counts():
1      164143
2       17907
3        2463
0         862
4          38
NaN         0

AdmDBP.value_counts():
1      172139
2       12444
3         783
0          47
NaN         0

BESTGA.value_counts():
39.00    24529
40.00    18045
38.00    13969
41.00     7722
37.00     6127
39.10     5983
39.30     4869
39.40     4293
39.60     3959
38.60     3552
40.10     3527
40.30     3096
36.00     2847
38.40     2842
40.40     2695
38.90     2638
38.30     2630
39.20     2371
40.60     2369
38.10     2312
39.70     2309
39.90     2273
38.70     2022
37.60     1630
39.50     1607
40.70     1572
40.90     1532
38.50     1484
37.40     1436
35.00     1407
41.10     1388
37.30     1333
37.10     1222
40.20     1175
38.20     1079
34.00     1065
38.86     1052
37.90     1024
39.14      951
38.71      948
37.70      918
38.57      853
42.00      847
39.29      845
39.43      843
38.43      818
41.30      809
39.86      768
36.60      765
39.71      761
39.57      757
38.29     

In [27]:
ty_PI_binned = pd.concat([bdf,pre_df], axis=1)
ty_PI_binned.insert(0, 'transfus_yes', ty_series)
print(ty_PI_binned.shape)
ty_PI_binned.head()

(185413, 193)


,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,0,1,0,77,2,7,1,38.2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,6,3,2,1,1,3,1,1,8,88,39.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,5,3,2,2,1,3,1,1,8,88,39.1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,11,2,3,2,1,4,77,1,8,88,38.4,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,11,1,3,2,2,4,77,1,8,88,39.6,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
tl_PI_binned = pd.concat([bdf,pre_df], axis=1)
tl_PI_binned.insert(0, 'trans_loss', tl_series)
print(tl_PI_binned.shape)
tl_PI_binned.head()

(185413, 193)


,trans_loss,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,Augment,BESTGA,breech,breech9,...,pre_diab,pre_PROM,Preeclampsia,prelaborCD,prim_hypo,renal_disease_comb,renal_disease9,Rhincompat,Smoke,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,0,0,8,8,8,0,1,0,77,2,7,1,38.2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
41-00003,0,0,6,3,2,1,1,3,1,1,8,88,39.0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00004,0,0,5,3,2,2,1,3,1,1,8,88,39.1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00005,1,0,11,2,3,2,1,4,77,1,8,88,38.4,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41-00006,0,0,11,1,3,2,2,4,77,1,8,88,39.6,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# Write out both Ty * Tl Binned Pre-Intra files
filename = 'MFM_CSL_d6_transfus_yes_Pre-Intra_binned_' + datestr + '.csv'
linkname = 'CSL_ty_PI_binned.csv'
# This version is not normalized
if out_flag:
    ty_PI_binned.to_csv(filename)
    if os.path.exists(linkname):
        os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print("out_flag False: File not written")

filename = 'MFM_CSL_d6_trans_loss_Pre-Intra_binned_' + datestr + '.csv'
linkname = 'CSL_tl_PI_binned.csv'
# This version is not normalized
if out_flag:
    tl_PI_binned.to_csv(filename)
    if os.path.exists(linkname):
        os.remove(linkname)
    os.symlink(filename, linkname)
else:
    print("out_flag False: File not written")